In [193]:
import time

import numpy as np
from numba import guvectorize, float32, float64, prange

def sgdpy():
    lr=1e-4
    mu=0.9
    weight_decay=0
    def aux_init(param):
        return np.zeros_like(param)
    def update_rule(x, dx, aux):
        x = (1-weight_decay) * x
        aux = mu * aux - lr * dx
        x = np.add(x, aux)
        return x, aux
    return aux_init, update_rule

def sgd(x, y, lr, mu, decay, out, init):
    init[:] = np.zeros_like(x)
    for i in prange(x.shape[0]):
        out[i] = (1-decay) * x[i]
        init[i] = mu * init[i] - lr * y[i]
        out[i] = np.add(out[i], init[i])

def gu_optimizer(func, *args, **kwargs):
    """
    The guvectorize decorated method, runs in nopython mode
    :param func: a loss error method
    :param target: choose between | 1: None -> serial | 2: 'parallel' -> parallel | execution method
    :return: the decorated loss, error function
    """
    kwargs_ = {k: v for k, v in kwargs.items() if v is not None}
    return guvectorize([(float32[:], float32[:], float32, float32, float32, float32[:], float32[:]),
                      (float64[:], float64[:], float64, float64, float64, float64[:], float64[:])],
                     '(n),(n),(),(),()->(n),(n)', nopython=True, fastmath=True, *args, **kwargs_)(func)

s_sgd = gu_optimizer(sgd)
p_sgd = gu_optimizer(sgd, target='parallel')

In [194]:
bigx = np.arange(0, 10000000, dtype=np.float32)
bigy = np.arange(10000000, 20000000, dtype=np.float32)

def ntime(func1):
    start = time.time()
    func1(bigx, bigy, 1e-4, 0.9, 0.0)
    end = time.time()
    print("Elapsed Numba Pre = %s" % (end - start))
    start = time.time()
    f2 = func1(bigx, bigy, 1e-4, 0.9, 0.0)
    end = time.time()
    print("Elapsed Numba Post = %s" % (end - start))
    return f2

def ptime(func1):
    start = time.time()
    init, update = func1()
    aux = init(bigx)
    f1 = update(bigx, bigy, aux)
    end = time.time()
    print("Elapsed PyTime = %s" % (end - start))
    return f1

In [ ]:
s = ntime(s_sgd)
p = ntime(p_sgd)
py = ptime(sgdpy)

In [ ]:
np.testing.assert_allclose(s, p)

In [195]:
np.testing.assert_allclose(s, p)

Elapsed Numba Pre = 0.03693389892578125
Elapsed Numba Post = 0.0359034538269043
Elapsed Numba Pre = 0.037859439849853516
Elapsed Numba Post = 0.03889584541320801
Elapsed PyTime = 0.055850982666015625


In [196]:
np.testing.assert_allclose(s, p)